In [2]:
setwd('C:/Users/iceca/Documents/Earthquake_Damage_Predictor/')
library(tidyverse)

In [29]:
loadPr<- modules::use('Helpers/Load_Preprocessed.R')
train <- loadPr$loadTrain()[[1]]
trainLab <- loadPr$loadTrain()[[2]]

val <- loadPr$loadVal()[[1]]
valLab <- loadPr$loadVal()[[2]]

loadRaw<- modules::use('Helpers/Load_Raw_Data.R')
test <- loadRaw$testVal()

In [40]:
removeId <- function(data) {
    data$X <- NULL
    data$building_id <- NULL
    return(data)
}
saveData <- function(tr, trLab, val, valLab, test, name) {
    tr <- removeId(tr)
    trLab <- removeId(trLab)
    val <- removeId(val)
    valLab <- removeId(valLab)
    #do not remove the ids of the test
    rootDir <- 'Further_Preprocess_Analysis/Data/'
    write.csv(tr, paste(rootDir, name, '_train.csv', sep=''))
    write.csv(trLab, paste(rootDir, name, '_train_lab.csv', sep=''))
    write.csv(val, paste(rootDir, name, '_val.csv', sep=''))
    write.csv(valLab, paste(rootDir, name, '_val_lab.csv', sep=''))
    write.csv(valLab, paste(rootDir, name, '_test.csv', sep=''))
}

In [41]:
names(train)
saveData(train, trainLab, val, valLab, test, 'original')

[1] "X"                                     
 [2] "building_id"                           
 [3] "geo_level_1_id"                        
 [4] "geo_level_2_id"                        
 [5] "geo_level_3_id"                        
 [6] "count_floors_pre_eq"                   
 [7] "age"                                   
 [8] "area_percentage"                       
 [9] "height_percentage"                     
[10] "land_surface_condition"                
[11] "foundation_type"                       
[12] "roof_type"                             
[13] "ground_floor_type"                     
[14] "other_floor_type"                      
[15] "position"                              
[16] "plan_configuration"                    
[17] "has_superstructure_adobe_mud"          
[18] "has_superstructure_mud_mortar_stone"   
[19] "has_superstructure_stone_flag"         
[20] "has_superstructure_cement_mortar_stone"
[21] "has_superstructure_mud_mortar_brick"   
[22] "has_superstructure_cement_mortar_brick"
[23] "has_superstructure_timber"             
[24] "has_superstructure_bamboo"             
[25] "has_superstructure_rc_non_engineered"  
[26] "has_superstructure_rc_engineered"      
[27] "has_superstructure_other"              
[28] "legal_ownership_status"                
[29] "count_families"                        
[30] "has_secondary_use"                     
[31] "has_secondary_use_agriculture"         
[32] "has_secondary_use_hotel"               
[33] "has_secondary_use_rental"              
[34] "has_secondary_use_institution"         
[35] "has_secondary_use_school"              
[36] "has_secondary_use_industry"            
[37] "has_secondary_use_health_post"         
[38] "has_secondary_use_gov_office"          
[39] "has_secondary_use_use_police"          
[40] "has_secondary_use_other"

### Making a dataset with more features

In [ ]:
featureEngineerAdd <- function(data) {
    data$has_superstructure_tree = 
        data$has_superstructure_bamboo | data$has_superstructure_timber
    data$has_superstructure_mortar = 
        data$has_superstructure_mud_mortar_stone | data$has_superstructure_cement_mortar_stone | data$has_superstructure_mud_mortar_brick | data$has_superstructure_cement_mortar_brick
    data$has_superstructure_cement = 
        data$has_superstructure_cement_mortar_stone | data$has_superstructure_timber
    data$has_superstructure_brick = 
        data$has_superstructure_mud_mortar_brick | data$has_superstructure_cement_mortar_brick
    data$has_superstructure_mud = 
        data$has_superstructure_adobe_mud | data$has_superstructure_mud_mortar_stone | data$has_superstructure_mud_mortar_brick
    data$Has_superstructure_concrete = 
        data$has_superstructure_rc_non_engineered | data$has_superstructure_rc_engineered 
    data$has_superstructure_stone = 
        data$has_superstructure_mud_mortar_stone | data$has_superstructure_stone_flag | data$has_superstructure_cement_mortar_stone
}

In [46]:

(val$has_superstructure_mud_mortar_stone | val$has_superstructure_cement_mortar_stone |
val$has_superstructure_mud_mortar_brick | val$has_superstructure_cement_mortar_brick) %>% sum()

[1] 2166

[1] 6680

[1] 6949